# Santander Product Recommendation

**Dataset description:**

This dataset contains 1.5 years of customer behavior data from Santander Bank, designed to predict the likelihood of customers purchasing new products. The data begins on 2015-01-28 and includes monthly records of products each customer holds, such as "credit card," "savings account," and others.

> See the following link for further details: [SPR dataset](https://www.kaggle.com/competitions/santander-product-recommendation/overview)

## Overview

This notebook is used to explore the Santander dataset to see what additional fields could be useful features in addition to the common set required.

> **Note:** Kernel - Python 3.11

## Access HuggingFace for Dataset access

### Sign in to your Hugging Face account

This will enable you to access the dataset and upload/share the model.

### Steps to get the `Access Token` from Hugging Face:

 - **Sign In or Sign Up:** If you don't have a Hugging Face account yet, you'll need to sign up. If you already have an account, sign in.

 - **Access Your Profile:** Once you're signed in, navigate to your profile settings. You can do this by clicking on your profile icon or username, usually located in the top-right corner of the Hugging Face website.
 
- **Navigate to Access Token Settings:** Within your profile settings, look for an option related to Access tokens. This is where you can manage and generate tokens.

- **Generate a New Token:** If you haven't generated a token before, you'll see a button (`New token`) to generate a new token. Click on this button. Please ensure you give the token `write` access

- **Name Your Token (Optional):** You may be prompted to give your token a name or description. This step is optional but can be helpful if you plan to generate multiple tokens for different purposes.

- **Copy Your Token:** Once your token is generated, you'll typically see it displayed on the screen. Make sure to copy the token and replace it in the `login` code below. 

In [182]:
# Log into Hugging Face
# Replace <access_token> with your access token

HUGGINGFACE_TOKEN = "<access_token>"
!huggingface-cli login --token $HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/verosha/.cache/huggingface/token
Login successful


In [183]:
# imports required

from huggingface_hub import hf_hub_download
from typing import List, Dict, Optional
import pandas as pd
import numpy as np


### Config

In [184]:
REPO_ID = "MelioAI/santander-product-recommendation"
HF_TRAIN_DATASET_NAME = "train_ver2.csv"
HF_TEST_DATASET_NAME = "test_ver2.csv"

### Helper Functions

The following cells define helper functions used throughout this notebook.

In [185]:
def update_gender_columns(df: pd.DataFrame, column_name: str = 'gender') -> pd.DataFrame:
    """
    Update gender columns in the DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the gender column.
    column_name (str): The name of the gender column. Default is 'gender'.

    Returns:
    pd.DataFrame: The DataFrame with updated gender values.
    """
    df[column_name].replace({'H': 'M', 'V': 'F'}, inplace=True)
    return df


def update_date_id(df: pd.DataFrame, column_name: str = 'date_id') -> pd.DataFrame:
    """
    Convert the date_id column in the DataFrame to YYYYMMDD format.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the date_id column.
    column_name (str): The name of the date_id column. Default is 'date_id'.

    Returns:
    pd.DataFrame: The DataFrame with date_id values in YYYYMMDD format.
    """
    # Convert column to string (if not already)
    df[column_name] = df[column_name].astype(str)

    # Convert ISO 8601 (YYYY-MM-DD) to YYYYMMDD
    df[column_name] = pd.to_datetime(df[column_name], format='%Y-%m-%d', errors='coerce').dt.strftime('%Y%m%d')

    return df

def map_products_with_numbers(owned_products: List[str], product_mapping: Dict[str, int]) -> List[int]:
    """
    Replace product names with corresponding numbers from the product_mapping dictionary.
    Return a list with -1 if the list is empty.

    Parameters:
    owned_products (List[str]): List of product names to be replaced.
    product_mapping (Dict[str, int]): Dictionary mapping product names to numbers.

    Returns:
    List[int]: List of corresponding product numbers, or [-1] if input list is empty.
    """
    if not owned_products:
        return [-1]
    return [product_mapping.get(product, -1) for product in owned_products]


def get_customer_details(df: pd.DataFrame, customer_id: int) -> Optional[pd.DataFrame]:
    """
    Retrieve the details of a customer given their customer ID.

    Parameters:
    df (pd.DataFrame): DataFrame containing customer data with columns 'cust_id', 'date_id', and 'owned_products'.
    customer_id (int): The ID of the customer to retrieve details for.

    Returns:
    Optional[pd.DataFrame]: A DataFrame containing the customer's details or an empty DataFrame if the customer ID is not found.
    """
    # Filter the DataFrame to get the row for the given customer_id
    customer_details = df[df['cust_id'] == customer_id]

    if not customer_details.empty:
        # Return the relevant columns as a DataFrame
        return customer_details[['cust_id', 'date_id', 'owned_products']]
    else:
        # Return an empty DataFrame if the customer ID is not found
        return pd.DataFrame(columns=['cust_id', 'date_id', 'owned_products'])


def check_columns(train_columns, test_columns):
    """
    Check if the columns in two sets are the same and print the common and different columns.

    Parameters:
    - train_columns: Set of columns from the training set
    - test_columns: Set of columns from the test set
    """
    if set(train_columns) == set(test_columns):
        print("Train and test sets have the same columns.")
        common_columns = set(train_columns)
        #print("Common columns:")
        #for col in common_columns:
        #    print(f"- {col}")
    else:
        different_columns_train = set(train_columns) - set(test_columns)
        different_columns_test = set(test_columns) - set(train_columns)

        print("Train and test sets have different columns.")

        if different_columns_train:
            print("Columns present in train but not in test:")
            for col in different_columns_train:
                print(f"- {col}")

        if different_columns_test:
            print("Columns present in test but not in train:")
            for col in different_columns_test:
                print(f"- {col}")



## Access Hugging Face

The cells below will use the Hugging Face Client Library to get train and test from the Santander Product Recommendation dataset

In [186]:
# NB: This may take a few seconds to a few minutes to run

ds_train = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename=HF_TRAIN_DATASET_NAME, repo_type="dataset")
)

/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_51658/4287082666.py:3: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  ds_train = pd.read_csv(


In [187]:
ds_test = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename=HF_TEST_DATASET_NAME, repo_type="dataset")
)

/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_51658/2311066649.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  ds_test = pd.read_csv(


In [188]:
# Data Profiling
# Getting an overview of the test and train dataframe’s structure

ds_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   fecha_dato             object 
 1   ncodpers               int64  
 2   ind_empleado           object 
 3   pais_residencia        object 
 4   sexo                   object 
 5   age                    object 
 6   fecha_alta             object 
 7   ind_nuevo              float64
 8   antiguedad             object 
 9   indrel                 float64
 10  ult_fec_cli_1t         object 
 11  indrel_1mes            object 
 12  tiprel_1mes            object 
 13  indresi                object 
 14  indext                 object 
 15  conyuemp               object 
 16  canal_entrada          object 
 17  indfall                object 
 18  tipodom                float64
 19  cod_prov               float64
 20  nomprov                object 
 21  ind_actividad_cliente  float64
 22  renta           

In [189]:
ds_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929615 entries, 0 to 929614
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   fecha_dato             929615 non-null  object 
 1   ncodpers               929615 non-null  int64  
 2   ind_empleado           929615 non-null  object 
 3   pais_residencia        929615 non-null  object 
 4   sexo                   929610 non-null  object 
 5   age                    929615 non-null  int64  
 6   fecha_alta             929615 non-null  object 
 7   ind_nuevo              929615 non-null  int64  
 8   antiguedad             929615 non-null  int64  
 9   indrel                 929615 non-null  int64  
 10  ult_fec_cli_1t         1683 non-null    object 
 11  indrel_1mes            929592 non-null  float64
 12  tiprel_1mes            929592 non-null  object 
 13  indresi                929615 non-null  object 
 14  indext                 929615 non-nu

In [190]:
# Check for nulls

ds_train.isnull().sum()

fecha_dato                      0
ncodpers                        0
ind_empleado                27734
pais_residencia             27734
sexo                        27804
age                             0
fecha_alta                  27734
ind_nuevo                   27734
antiguedad                      0
indrel                      27734
ult_fec_cli_1t           13622516
indrel_1mes                149781
tiprel_1mes                149781
indresi                     27734
indext                      27734
conyuemp                 13645501
canal_entrada              186126
indfall                     27734
tipodom                     27735
cod_prov                    93591
nomprov                     93591
ind_actividad_cliente       27734
renta                     2794375
segmento                   189368
ind_ahor_fin_ult1               0
ind_aval_fin_ult1               0
ind_cco_fin_ult1                0
ind_cder_fin_ult1               0
ind_cno_fin_ult1                0
ind_ctju_fin_u

In [191]:
# Check for nulls

ds_test.isnull().sum()

fecha_dato                    0
ncodpers                      0
ind_empleado                  0
pais_residencia               0
sexo                          5
age                           0
fecha_alta                    0
ind_nuevo                     0
antiguedad                    0
indrel                        0
ult_fec_cli_1t           927932
indrel_1mes                  23
tiprel_1mes                  23
indresi                       0
indext                        0
conyuemp                 929511
canal_entrada              2081
indfall                       0
tipodom                       0
cod_prov                   3996
nomprov                    3996
ind_actividad_cliente         0
renta                         0
segmento                   2248
dtype: int64

In [192]:
# Generate a detailed statistical summary of the train DataFrame, including both numeric and categorical columns
# Transpose for easier readability and analysis
ds_train.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
fecha_dato,13647309,17,2016-05-28,931453,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ncodpers,13647309.0,NaN,NaN,NaN,834904.211501,431565.025784,15889.0,452813.0,931893.0,1199286.0,1553689.0
ind_empleado,13619575,5,N,13610977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pais_residencia,13619575,118,ES,13553710,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sexo,13619505,2,V,7424252,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,13647309.0,235.0,23.0,542682.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fecha_alta,13619575,6756,2014-07-28,57389,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ind_nuevo,13619575.0,NaN,NaN,NaN,0.059562,0.236673,0.0,0.0,0.0,0.0,1.0
antiguedad,13647309.0,507.0,0.0,134335.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
indrel,13619575.0,NaN,NaN,NaN,1.178399,4.177469,1.0,1.0,1.0,1.0,99.0


In [193]:
# Convert column names from Spanish to more readable English names

col_names = {"fecha_dato": "date_id","ncodpers":"cust_id", "ind_empleado":"emp_index","pais_residencia":"cust_country_res",
            "sexo":"gender","fecha_alta":"cust_start_date_first_holder_contract","ind_nuevo":"new_cust_index","antiguedad":"cust_seniority",
            "indrel":"cust_primary_type","ult_fec_cli_1t":"cust_last_primary_date","indrel_1mes":"cust_type_at_start_month",
            "tiprel_1mes":"cust_rel_type_at_start_month","indresi":"residence_index","indext":"foreigner_index",
            "conyuemp":"spouse_index","canal_entrada":"channel_joined", "indfall":"deceased_index", "tipodom":"address_type",
            "cod_prov":"province","nomprov":"province_name", "ind_actividad_cliente":"activity_index","renta":"gross_income",
            "segmento":"cust_category", "ind_ahor_fin_ult1":"savings_acc", "ind_aval_fin_ult1":"guarantees",
            "ind_cco_fin_ult1":"current_acc", "ind_cder_fin_ult1":"derivada_acc", "ind_cno_fin_ult1":"payroll_acc",
            "ind_ctju_fin_ult1":"jnr_acc", "ind_ctma_fin_ult1":"más_particular_acc", "ind_ctop_fin_ult1":"particular_account",
            "ind_ctpp_fin_ult1":"particular_plus_account", "ind_deco_fin_ult1":"short_term_deposits",
            "ind_deme_fin_ult1":"medium_term_deposits", "ind_dela_fin_ult1":"long_term_deposits", "ind_ecue_fin_ult1":"e_acc",
            "ind_fond_fin_ult1":"funds","ind_hip_fin_ult1":"mortgage", "ind_plan_fin_ult1":"pensions_plan", "ind_pres_fin_ult1":"loans",
            "ind_reca_fin_ult1":"taxes", "ind_tjcr_fin_ult1":"credit_card", "ind_valo_fin_ult1":"securities", "ind_viv_fin_ult1":"home_acc",
            "ind_nomina_ult1":"payroll", "ind_nom_pens_ult1": "pensions", "ind_recibo_ult1":"direct_debit"}

In [194]:
# Rename columns in ds_train and ds_test according to the col_names dictionary,
ds_train_renamed = ds_train.rename(col_names, axis = 1, inplace = False)
ds_test_renamed = ds_test.rename(col_names, axis = 1, inplace = False)

# sample output of the renamed columns for ds_train
ds_train_renamed.sample(3)

,date_id,cust_id,emp_index,cust_country_res,gender,age,cust_start_date_first_holder_contract,new_cust_index,cust_seniority,cust_primary_type,...,mortgage,pensions_plan,loans,taxes,credit_card,securities,home_acc,payroll,pensions,direct_debit
2342650,2015-04-28,1317366,N,ES,H,22,2014-09-25,0.0,10,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
7478407,2015-11-28,75885,N,ES,V,62,1997-11-10,0.0,216,1.0,...,0,0,0,0,0,1,0,0.0,0.0,1
12745383,2016-05-28,723927,N,ES,V,42,2007-10-07,0.0,103,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [195]:
# check the updated column names
ds_train_renamed.columns

Index(['date_id', 'cust_id', 'emp_index', 'cust_country_res', 'gender', 'age',
       'cust_start_date_first_holder_contract', 'new_cust_index',
       'cust_seniority', 'cust_primary_type', 'cust_last_primary_date',
       'cust_type_at_start_month', 'cust_rel_type_at_start_month',
       'residence_index', 'foreigner_index', 'spouse_index', 'channel_joined',
       'deceased_index', 'address_type', 'province', 'province_name',
       'activity_index', 'gross_income', 'cust_category', 'savings_acc',
       'guarantees', 'current_acc', 'derivada_acc', 'payroll_acc', 'jnr_acc',
       'más_particular_acc', 'particular_account', 'particular_plus_account',
       'short_term_deposits', 'medium_term_deposits', 'long_term_deposits',
       'e_acc', 'funds', 'mortgage', 'pensions_plan', 'loans', 'taxes',
       'credit_card', 'securities', 'home_acc', 'payroll', 'pensions',
       'direct_debit'],
      dtype='object')

In [196]:
ds_train_renamed.dtypes

date_id                                   object
cust_id                                    int64
emp_index                                 object
cust_country_res                          object
gender                                    object
age                                       object
cust_start_date_first_holder_contract     object
new_cust_index                           float64
cust_seniority                            object
cust_primary_type                        float64
cust_last_primary_date                    object
cust_type_at_start_month                  object
cust_rel_type_at_start_month              object
residence_index                           object
foreigner_index                           object
spouse_index                              object
channel_joined                            object
deceased_index                            object
address_type                             float64
province                                 float64
province_name       

In [197]:
# Perform data type conversions on columns

ds_train_renamed.age = pd.to_numeric(ds_train_renamed.age, errors='coerce')
ds_train_renamed.gross_income = pd.to_numeric(ds_train_renamed.gross_income, errors='coerce')
ds_train_renamed.cust_seniority = pd.to_numeric(ds_train_renamed.cust_seniority, errors='coerce')
ds_train_renamed.cust_start_date_first_holder_contract = pd.to_datetime(ds_train_renamed.cust_start_date_first_holder_contract, errors = 'coerce')
ds_train_renamed['date_id'] = pd.to_datetime(ds_train_renamed['date_id'])

ds_test_renamed.age = pd.to_numeric(ds_test_renamed.age, errors='coerce')
ds_test_renamed.gross_income = pd.to_numeric(ds_test_renamed.gross_income, errors='coerce')
ds_test_renamed.cust_seniority = pd.to_numeric(ds_test_renamed.cust_seniority, errors='coerce')
ds_test_renamed.cust_start_date_first_holder_contract = pd.to_datetime(ds_test_renamed.cust_start_date_first_holder_contract, errors = 'coerce')
ds_test_renamed['date_id'] = pd.to_datetime(ds_test_renamed['date_id'])


In [198]:
# Get the percentage of missing values in each column
ds_train_renamed.isnull().sum()/ds_train_renamed.shape[0] * 100
# ds_test_renamed.isnull().sum()/ds_train_renamed.shape[0] * 100

date_id                                   0.000000
cust_id                                   0.000000
emp_index                                 0.203220
cust_country_res                          0.203220
gender                                    0.203732
age                                       0.203220
cust_start_date_first_holder_contract     0.203220
new_cust_index                            0.203220
cust_seniority                            0.203220
cust_primary_type                         0.203220
cust_last_primary_date                   99.818330
cust_type_at_start_month                  1.097513
cust_rel_type_at_start_month              1.097513
residence_index                           0.203220
foreigner_index                           0.203220
spouse_index                             99.986752
channel_joined                            1.363829
deceased_index                            0.203220
address_type                              0.203227
province                       

In [199]:
# According to the percentage of missing values, we will perform appropriate imputation for each column

# Imputation 1: Drop columns which have > 99% missing values
columns_to_drop = ['cust_last_primary_date', 'spouse_index']

# Drop columns if they exist in the DataFrame
ds_train_renamed.drop(columns=[col for col in columns_to_drop if col in ds_train_renamed.columns], axis=1, inplace=True)
ds_test_renamed.drop(columns=[col for col in columns_to_drop if col in ds_test_renamed.columns], axis=1, inplace=True)

In [200]:
# Imputation 2: Missing values that have <10% are imputed with the most common value (mode) in each column

# Columns <10% missing values
cols = ['emp_index','cust_country_res','cust_start_date_first_holder_contract','new_cust_index',
        'cust_primary_type',"cust_type_at_start_month", "cust_rel_type_at_start_month", "province","province_name",
        "activity_index","channel_joined","cust_category"]

# Impute missing values with the most common value (mode) in each column
for i in cols:
    if i in ds_train_renamed.columns and i in ds_test_renamed.columns:
        # Impute with mode for train DataFrame
        mode_value_train = ds_train_renamed[i].mode()[0]  # Use mode()[0] for the first mode
        ds_train_renamed[i].fillna(mode_value_train, inplace=True)

        # Impute with mode for test DataFrame
        mode_value_test = ds_test_renamed[i].mode()[0]  # Use mode()[0] for the first mode
        ds_test_renamed[i].fillna(mode_value_test, inplace=True)

# For features with missing value accounts for over 10%, impute their missing values based on the mean
# For gross_income, impute the missing values using the mean value
ds_train_renamed['gross_income'].fillna(ds_train_renamed['gross_income'].mean(), inplace=True)
ds_test_renamed['gross_income'].fillna(ds_test_renamed['gross_income'].mean(), inplace=True)

/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_51658/3826259992.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ds_train_renamed[i].fillna(mode_value_train, inplace=True)
/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_51658/3826259992.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

In [201]:
# Get the percentage of missing values in each column
ds_train_renamed.isnull().sum()/ds_train_renamed.shape[0] * 100

date_id                                  0.000000
cust_id                                  0.000000
emp_index                                0.000000
cust_country_res                         0.000000
gender                                   0.203732
age                                      0.203220
cust_start_date_first_holder_contract    0.000000
new_cust_index                           0.000000
cust_seniority                           0.203220
cust_primary_type                        0.000000
cust_type_at_start_month                 0.000000
cust_rel_type_at_start_month             0.000000
residence_index                          0.203220
foreigner_index                          0.203220
channel_joined                           0.000000
deceased_index                           0.203220
address_type                             0.203227
province                                 0.000000
province_name                            0.000000
activity_index                           0.000000


In [202]:
# Get the percentage of missing values in each column for test
ds_test_renamed.isnull().sum()/ds_test_renamed.shape[0] * 100

date_id                                  0.000000
cust_id                                  0.000000
emp_index                                0.000000
cust_country_res                         0.000000
gender                                   0.000538
age                                      0.000000
cust_start_date_first_holder_contract    0.000000
new_cust_index                           0.000000
cust_seniority                           0.000000
cust_primary_type                        0.000000
cust_type_at_start_month                 0.000000
cust_rel_type_at_start_month             0.000000
residence_index                          0.000000
foreigner_index                          0.000000
channel_joined                           0.000000
deceased_index                           0.000000
address_type                             0.000000
province                                 0.000000
province_name                            0.000000
activity_index                           0.000000


In [203]:
# Drop rows where 'payroll' or 'pensions' is NaN in the training data
df_train_cleaned = ds_train_renamed.dropna(subset=['payroll', 'pensions'])


In [204]:
unique_values = df_train_cleaned['deceased_index'].unique()
print(unique_values)

['N' nan 'S']


In [205]:
#df_train_cleaned['cust_id','emp_index'].head(10)
df_train_cleaned[['cust_id', 'payroll_acc']].sample(5)

,cust_id,payroll_acc
3273186,1020848,0
11500034,679191,0
12802784,503192,0
12014083,843822,0
6240537,1363650,0


### Features to consider

- emp_index: Employment status might affect financial needs.
 - A active, B ex employed, F filial, N not employee, P pasive

- channel_joined: The channel through which the customer joined might influence their behavior.

- cust_primary_type: This will indicate the main relationship type the customer has with the bank.
    - 1 (First/Primary), 99 (Primary customer during the month but not at the end of the month)

- cust_rel_type_at_start_month: The type of relationship at the start of the month could influence product needs.
    - Customer relation type at the beginning of the month, A (active), I (inactive), P (former customer),R (Potential)

- activity_index: Indicates whether the customer is active, which is crucial for determining product needs.
    - activity_index: Indicates whether the customer is active, which is crucial for determining product needs.

- gross_income: Income level can influence the types of products a customer might need.
    - gross income of the household

- cust_category: Customer segmentation can indicate different needs and preferences.
    - segmentation: 01 - VIP, 02 - Individuals 03 - college graduated

- All banking products

### Create Dataset with the key features
- Pre-process features selected 
- Check for nulls 
- Check that train and test have the same columns

In [206]:
df_train_cleaned.columns

Index(['date_id', 'cust_id', 'emp_index', 'cust_country_res', 'gender', 'age',
       'cust_start_date_first_holder_contract', 'new_cust_index',
       'cust_seniority', 'cust_primary_type', 'cust_type_at_start_month',
       'cust_rel_type_at_start_month', 'residence_index', 'foreigner_index',
       'channel_joined', 'deceased_index', 'address_type', 'province',
       'province_name', 'activity_index', 'gross_income', 'cust_category',
       'savings_acc', 'guarantees', 'current_acc', 'derivada_acc',
       'payroll_acc', 'jnr_acc', 'más_particular_acc', 'particular_account',
       'particular_plus_account', 'short_term_deposits',
       'medium_term_deposits', 'long_term_deposits', 'e_acc', 'funds',
       'mortgage', 'pensions_plan', 'loans', 'taxes', 'credit_card',
       'securities', 'home_acc', 'payroll', 'pensions', 'direct_debit'],
      dtype='object')

In [207]:
# Define the columns to retain in the new DataFrame
retained_columns = [
    'cust_id', 'date_id', 'age', 'gender', 'emp_index', 'channel_joined',
    'cust_primary_type', 'cust_rel_type_at_start_month', 'activity_index',
    'gross_income', 'cust_category', 'deceased_index'
]

# Define the product columns
product_columns = [
    'savings_acc', 'guarantees', 'current_acc', 'derivada_acc', 'payroll_acc', 'jnr_acc',
    'más_particular_acc', 'particular_account', 'particular_plus_account', 'short_term_deposits',
    'medium_term_deposits', 'long_term_deposits', 'e_acc', 'funds', 'mortgage', 'pensions_plan',
    'loans', 'taxes', 'credit_card', 'securities', 'home_acc', 'payroll', 'pensions', 'direct_debit'
]

# Combine all the columns to include in the new DataFrame
all_columns = retained_columns + product_columns

# Create the new DataFrame with the selected columns
df_train_subset_cols = df_train_cleaned[all_columns]

# Display the first few rows of the new DataFrame
df_train_subset_cols.head(5)

,cust_id,date_id,age,gender,emp_index,channel_joined,cust_primary_type,cust_rel_type_at_start_month,activity_index,gross_income,...,mortgage,pensions_plan,loans,taxes,credit_card,securities,home_acc,payroll,pensions,direct_debit
0,1375586,2015-01-28,35.0,H,N,KHL,1.0,A,1.0,87218.100000,...,0,0,0,0,0,0,0,0.0,0.0,0
1,1050611,2015-01-28,23.0,V,N,KHE,1.0,I,0.0,35548.740000,...,0,0,0,0,0,0,0,0.0,0.0,0
2,1050612,2015-01-28,23.0,V,N,KHE,1.0,I,0.0,122179.110000,...,0,0,0,0,0,0,0,0.0,0.0,0
3,1050613,2015-01-28,22.0,H,N,KHD,1.0,I,0.0,119775.540000,...,0,0,0,0,0,0,0,0.0,0.0,0
4,1050614,2015-01-28,23.0,V,N,KHE,1.0,A,1.0,134254.318238,...,0,0,0,0,0,0,0,0.0,0.0,0


In [208]:
# Get the percentage of missing values in each column
df_train_subset_cols.isnull().sum()/df_train_subset_cols.shape[0] * 100

cust_id                         0.000000
date_id                         0.000000
age                             0.087211
gender                          0.087725
emp_index                       0.000000
channel_joined                  0.000000
cust_primary_type               0.000000
cust_rel_type_at_start_month    0.000000
activity_index                  0.000000
gross_income                    0.000000
cust_category                   0.000000
deceased_index                  0.087211
savings_acc                     0.000000
guarantees                      0.000000
current_acc                     0.000000
derivada_acc                    0.000000
payroll_acc                     0.000000
jnr_acc                         0.000000
más_particular_acc              0.000000
particular_account              0.000000
particular_plus_account         0.000000
short_term_deposits             0.000000
medium_term_deposits            0.000000
long_term_deposits              0.000000
e_acc           

In [209]:
# Since age, gender and deceased_index have very low percentages of missing values (0.087211%)
# Remove rows with missing values in 'age', 'gender' 'deceased_index'

df_train_subset_cols_cleaned = df_train_subset_cols.dropna(subset=['age', 'deceased_index', 'gender'])
df_test_subset_cols_cleaned = ds_test_renamed.dropna(subset=['gender'])


In [210]:
#  Check the unique values of each column in a DataFrame
for column in df_train_subset_cols_cleaned.columns:
    unique_values = df_train_subset_cols_cleaned[column].unique()
    print(f"Unique values in {column}: {unique_values}\n")


Unique values in cust_id: [1375586 1050611 1050612 ... 1173729 1164094 1550586]

Unique values in date_id: <DatetimeArray>
['2015-01-28 00:00:00', '2015-02-28 00:00:00', '2015-03-28 00:00:00',
 '2015-04-28 00:00:00', '2015-05-28 00:00:00', '2015-06-28 00:00:00',
 '2015-07-28 00:00:00', '2015-08-28 00:00:00', '2015-09-28 00:00:00',
 '2015-10-28 00:00:00', '2015-11-28 00:00:00', '2015-12-28 00:00:00',
 '2016-01-28 00:00:00', '2016-02-28 00:00:00', '2016-03-28 00:00:00',
 '2016-04-28 00:00:00', '2016-05-28 00:00:00']
Length: 17, dtype: datetime64[ns]

Unique values in age: [ 35.  23.  22.  24.  65.  28.  25.  26.  53.  27.  32.  37.  31.  39.
  63.  33.  55.  42.  58.  38.  50.  30.  45.  44.  36.  29.  60.  57.
  67.  47.  34.  48.  46.  54.  84.  15.  12.   8.   6.  83.  40.  77.
  69.  52.  59.  43.  10.   9.  49.  41.  51.  78.  16.  11.  73.  62.
  66.  17.  68.  82.  95.  96.  56.  61.  79.  72.  14.  19.  13.  86.
  64.  20.  89.  71.   7.  70.  74.  21.  18.  75.   4.  80.  81.   

In [211]:
#  Check the unique values of each column in a DataFrame
for column in df_test_subset_cols_cleaned.columns:
    unique_values = df_test_subset_cols_cleaned[column].unique()
    print(f"Unique values in {column}: {unique_values}\n")

Unique values in date_id: <DatetimeArray>
['2016-06-28 00:00:00']
Length: 1, dtype: datetime64[ns]

Unique values in cust_id: [  15889 1170544 1170545 ...  660240  660243  660248]

Unique values in emp_index: ['F' 'N' 'A' 'B' 'S']

Unique values in cust_country_res: ['ES' 'CH' 'DE' 'GB' 'BE' 'DJ' 'IE' 'QA' 'US' 'VE' 'DO' 'SE' 'AR' 'CA'
 'PL' 'CN' 'CM' 'FR' 'AT' 'RO' 'LU' 'PT' 'CL' 'IT' 'MR' 'MX' 'SN' 'BR'
 'CO' 'PE' 'RU' 'LT' 'EE' 'MA' 'HN' 'BG' 'NO' 'GT' 'UA' 'NL' 'GA' 'IL'
 'JP' 'EC' 'IN' 'DZ' 'ET' 'SA' 'HU' 'JM' 'CI' 'CU' 'BO' 'TG' 'TN' 'NG'
 'AU' 'GR' 'DK' 'LB' 'UY' 'TH' 'SG' 'MD' 'SK' 'AD' 'BY' 'HK' 'HR' 'EG'
 'GQ' 'PR' 'ZA' 'PA' 'KE' 'TR' 'FI' 'BA' 'SV' 'PY' 'PK' 'KR' 'AO' 'GN'
 'IS' 'TW' 'MK' 'VN' 'CZ' 'CR' 'MZ' 'MT' 'LY' 'GH' 'KH' 'AE' 'RS' 'OM'
 'GE' 'NI' 'GI' 'NZ' 'MM' 'PH' 'KW' 'BM' 'CG' 'ML' 'AL' 'ZW' 'CF' 'GM'
 'CD' 'BZ' 'KZ' 'GW' 'SL' 'LV']

Unique values in gender: ['V' 'H']

Unique values in age: [ 56  36  22  51  41  33  23  43  63  62  32  58  71  31  30  59  45  37
 

In [212]:
# Step 1: Update gender values in the data as it's currently reflected as "H" and "V" instead of "F" and "M"
df_train_subset_cols_cleaned = update_gender_columns(df_train_subset_cols_cleaned)
df_test_subset_cols_cleaned = update_gender_columns(df_test_subset_cols_cleaned)

# sample output of the gender update for ds_train
# df_train_subset_cols_cleaned.sample(5)
df_test_subset_cols_cleaned.sample(5)

/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_51658/1269225786.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].replace({'H': 'M', 'V': 'F'}, inplace=True)
/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_51658/1269225786.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name].replace({'H

,date_id,cust_id,emp_index,cust_country_res,gender,age,cust_start_date_first_holder_contract,new_cust_index,cust_seniority,cust_primary_type,...,residence_index,foreigner_index,channel_joined,deceased_index,address_type,province,province_name,activity_index,gross_income,cust_category
96627,2016-06-28,1196585,N,ES,M,22,2013-10-11,0,32,1,...,S,N,KHE,N,1,30.0,MURCIA,0,59582.58,03 - UNIVERSITARIO
405718,2016-06-28,1390357,N,ES,M,46,2015-04-01,0,14,1,...,S,N,KHN,N,1,47.0,VALLADOLID,1,118077.33,02 - PARTICULARES
642028,2016-06-28,217778,N,ES,F,59,2001-01-24,0,185,1,...,S,N,KAT,N,1,12.0,CASTELLON,1,48531.57,01 - TOP
741193,2016-06-28,812874,N,ES,M,63,2008-11-02,0,91,1,...,S,N,KFC,N,1,28.0,MADRID,1,115076.43,02 - PARTICULARES
158582,2016-06-28,977104,N,ES,F,24,2011-11-11,0,55,1,...,S,N,KHE,N,1,9.0,BURGOS,0,49228.50,03 - UNIVERSITARIO


In [213]:
# Step 2: Update date_id format to YYYYMMDD (20150328)
df_train_subset_cols_cleaned = update_date_id(df_train_subset_cols_cleaned)
df_test_subset_cols_cleaned = update_date_id(df_test_subset_cols_cleaned)

# sample output of the date format update for ds_train
# df_train_subset_cols_cleaned.sample(5)
df_test_subset_cols_cleaned.sample(5)

/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_51658/1269225786.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].astype(str)
/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_51658/1269225786.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = pd.to_datetime(df[column_name], format='%Y-%m-%d', errors='coerce').dt.strftime('%Y%m%d')
/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_51658/1269225786.py:28: SettingWithCopyWa

,date_id,cust_id,emp_index,cust_country_res,gender,age,cust_start_date_first_holder_contract,new_cust_index,cust_seniority,cust_primary_type,...,residence_index,foreigner_index,channel_joined,deceased_index,address_type,province,province_name,activity_index,gross_income,cust_category
262113,20160628,1474488,N,ES,M,22,2015-10-09,0,8,1,...,S,N,KHQ,N,1,8.0,BARCELONA,0,134087.870595,03 - UNIVERSITARIO
922696,20160628,655380,N,ES,F,37,2006-11-09,0,115,1,...,S,N,KFC,N,1,28.0,MADRID,0,72788.010000,02 - PARTICULARES
720449,20160628,858662,N,ES,M,70,2015-10-13,0,8,1,...,S,N,KHM,N,1,41.0,SEVILLA,1,134087.870595,02 - PARTICULARES
467862,20160628,340944,N,ES,M,66,2002-03-11,0,171,1,...,S,N,KAT,N,1,28.0,MADRID,1,47728.860000,02 - PARTICULARES
228598,20160628,1064030,N,ES,F,31,2012-09-20,0,45,1,...,S,N,KHE,N,1,41.0,SEVILLA,1,106078.590000,03 - UNIVERSITARIO


In [214]:
# Step 3: Update the values in the cust_category column from ['02 - PARTICULARES', '03 - UNIVERSITARIO', '01 - TOP'] to [2, 3, 1]

# Define the mapping from the original values to the new values
category_mapping = {
    '02 - PARTICULARES': 2,
    '03 - UNIVERSITARIO': 3,
    '01 - TOP': 1
}

# Replace the values in the 'cust_category' column
df_train_subset_cols_cleaned['cust_category'] = df_train_subset_cols_cleaned['cust_category'].replace(category_mapping)
df_test_subset_cols_cleaned['cust_category'] = df_test_subset_cols_cleaned['cust_category'].replace(category_mapping)

# Verify the changes
df_train_subset_cols_cleaned['cust_category'].unique()

/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_51658/2973078221.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train_subset_cols_cleaned['cust_category'] = df_train_subset_cols_cleaned['cust_category'].replace(category_mapping)
/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_51658/2973078221.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_subset_cols_cleaned['cust_category'] = df_train_subset_cols_cleaned['cust_category'].replace(category_mapping)
/var/folders/py/00rrgyx109

array([2, 3, 1])

In [215]:
# Drop deceased_index
df_train_subset_cols_cleaned = df_train_subset_cols_cleaned.drop(columns=['deceased_index'])
df_test_subset_cols_cleaned = df_test_subset_cols_cleaned.drop(columns=['deceased_index'])

In [216]:
# Check that the train and test set have the same columns present
check_columns(df_train_subset_cols_cleaned.columns, df_test_subset_cols_cleaned.columns)

Train and test sets have different columns.
Columns present in train but not in test:
- home_acc
- funds
- jnr_acc
- pensions_plan
- e_acc
- más_particular_acc
- loans
- mortgage
- payroll
- guarantees
- securities
- particular_account
- current_acc
- payroll_acc
- derivada_acc
- long_term_deposits
- short_term_deposits
- direct_debit
- taxes
- credit_card
- medium_term_deposits
- particular_plus_account
- pensions
- savings_acc
Columns present in test but not in train:
- cust_type_at_start_month
- address_type
- cust_start_date_first_holder_contract
- cust_country_res
- cust_seniority
- new_cust_index
- province
- foreigner_index
- province_name
- residence_index


In [217]:
# Products not present in test set
# Calculate the ownership probability for each product
ownership_probabilities = {col: df_train_subset_cols_cleaned[col].mean() for col in product_columns}

print("Ownership Probabilities Based on Training Data:")
print(ownership_probabilities)

Ownership Probabilities Based on Training Data:
{'savings_acc': 0.00010250168731287568, 'guarantees': 2.320238767254206e-05, 'current_acc': 0.6562846016619959, 'derivada_acc': 0.000394440590433215, 'payroll_acc': 0.08098925582600207, 'jnr_acc': 0.00947538520369053, 'más_particular_acc': 0.00971078664317841, 'particular_account': 0.12918318490658248, 'particular_plus_account': 0.04336900724913079, 'short_term_deposits': 0.0017499446373408067, 'medium_term_deposits': 0.0016640370627304451, 'long_term_deposits': 0.04304307244255353, 'e_acc': 0.08282503461267579, 'funds': 0.018514624259359225, 'mortgage': 0.005878280861672064, 'pensions_plan': 0.009185355357783755, 'loans': 0.00253486085322522, 'taxes': 0.051892140029640314, 'credit_card': 0.044463484434722284, 'securities': 0.02564921161811102, 'home_acc': 0.003853652261410435, 'payroll': 0.05475734120608948, 'pensions': 0.05946419519140795, 'direct_debit': 0.12811146955699887}


In [218]:
# Approach for Imputing Product Ownership (test set)
# Ownership probabilities based on training data
product_probabilities = {
    'savings_acc': 0.00010250168731287568,
    'guarantees': 2.320238767254206e-05,
    'current_acc': 0.6562846016619959,
    'derivada_acc': 0.000394440590433215,
    'payroll_acc': 0.08098925582600207,
    'jnr_acc': 0.00947538520369053,
    'más_particular_acc': 0.00971078664317841,
    'particular_account': 0.12918318490658248,
    'particular_plus_account': 0.04336900724913079,
    'short_term_deposits': 0.0017499446373408067,
    'medium_term_deposits': 0.0016640370627304451,
    'long_term_deposits': 0.04304307244255353,
    'e_acc': 0.08282503461267579,
    'funds': 0.018514624259359225,
    'mortgage': 0.005878280861672064,
    'pensions_plan': 0.009185355357783755,
    'loans': 0.00253486085322522,
    'taxes': 0.051892140029640314,
    'credit_card': 0.044463484434722284,
    'securities': 0.02564921161811102,
    'home_acc': 0.003853652261410435,
    'payroll': 0.05475734120608948,
    'pensions': 0.05946419519140795,
    'direct_debit': 0.12811146955699887
}

# Add missing product columns to the test DataFrame with default NaN values
for product in product_probabilities:
    if product not in df_test_subset_cols_cleaned.columns:
        df_test_subset_cols_cleaned[product] = np.nan

# Ensure the size parameter matches the number of rows in the DataFrame
num_rows = len(df_test_subset_cols_cleaned)

# Fill product columns based on probabilities
for product, prob in product_probabilities.items():
    if product in df_test_subset_cols_cleaned.columns:
        # Ensure size matches the number of rows
        df_test_subset_cols_cleaned[product] = np.random.binomial(1, prob, size=num_rows)

# Ensure all columns are present and in the correct order
required_columns = [
    'cust_id', 'date_id', 'age', 'gender', 'emp_index', 'channel_joined',
    'cust_primary_type', 'cust_rel_type_at_start_month', 'activity_index',
    'gross_income', 'cust_category'
] + list(product_probabilities.keys())

# Ensure the DataFrame has the required columns
for col in required_columns:
    if col not in df_test_subset_cols_cleaned.columns:
        df_test_subset_cols_cleaned[col] = np.nan

# Reorder columns to match the required list
df_test_subset_cols_cleaned = df_test_subset_cols_cleaned[required_columns]


In [219]:
df_test_subset_cols_cleaned.sample(3)

,cust_id,date_id,age,gender,emp_index,channel_joined,cust_primary_type,cust_rel_type_at_start_month,activity_index,gross_income,...,mortgage,pensions_plan,loans,taxes,credit_card,securities,home_acc,payroll,pensions,direct_debit
309049,1551378,20160628,24,M,N,KHM,1,A,0,134087.870595,...,0,0,0,0,0,0,0,0,0,0
595564,122958,20160628,51,F,N,KAT,1,A,1,86092.200000,...,0,0,0,0,0,0,0,0,0,0
528319,454206,20160628,67,M,N,KAT,1,A,1,211687.230000,...,0,0,0,0,0,0,0,0,0,1


### Add a new column
 - owned_products
    - list of products owned by the customer for the latest date_id
    - format: list e.g: ['current_acc', 'mortgage']
    - map product names to numbers eg: savings_acc = 0 and no products owned will be -1

In [220]:
# Create the 'owned_products' column
# Check if the value is not equal to 0 for the products.
# If the value is not 0, it includes the column name (product) in the list.
# NB: This may take a few minutes to finish running
df_train_subset_cols_cleaned['owned_products'] = df_train_subset_cols_cleaned[product_columns].apply(lambda row: [col for col in product_columns if row[col] != 0], axis=1)
df_test_subset_cols_cleaned['owned_products'] = df_test_subset_cols_cleaned[product_columns].apply(lambda row: [col for col in product_columns if row[col] != 0], axis=1)

In [221]:
# Check that the train and test set have the same columns present
check_columns(df_train_subset_cols_cleaned.columns, df_test_subset_cols_cleaned.columns)

Train and test sets have the same columns.


In [222]:
df_train_subset_cols_cleaned.sample(4)

,cust_id,date_id,age,gender,emp_index,channel_joined,cust_primary_type,cust_rel_type_at_start_month,activity_index,gross_income,...,pensions_plan,loans,taxes,credit_card,securities,home_acc,payroll,pensions,direct_debit,owned_products
6544139,1414574,20151028,19.0,F,N,KHQ,1.0,A,1.0,83738.43,...,0,0,0,0,0,0,0.0,0.0,0,[current_acc]
5469790,517787,20150928,45.0,F,N,KFC,1.0,A,1.0,68446.26,...,0,0,0,0,0,0,0.0,0.0,0,"[current_acc, particular_account]"
9800514,408804,20160128,48.0,F,N,KAT,1.0,A,1.0,75918.54,...,0,0,1,1,0,0,1.0,1.0,1,"[payroll_acc, e_acc, mortgage, taxes, credit_c..."
3821067,673747,20150728,44.0,M,N,KFC,1.0,I,0.0,141955.29,...,0,0,0,0,0,0,0.0,0.0,0,[current_acc]


In [223]:
# Step 4: Map product names to numbers eg: savings_acc = 0 and no products owned will be -1

# Create a dictionary to map product names to numbers
product_mapping = {product: idx for idx, product in enumerate(product_columns)}

In [224]:
# Replace product names with numbers and ensure non-empty lists
df_train_subset_cols_cleaned['owned_products'] = df_train_subset_cols_cleaned['owned_products'].apply(lambda x: map_products_with_numbers(x, product_mapping))
df_test_subset_cols_cleaned['owned_products'] = df_test_subset_cols_cleaned['owned_products'].apply(lambda x: map_products_with_numbers(x, product_mapping))

In [225]:
# View a small sample of the selected columns with the mapping update

df_train_subset_cols_cleaned.sample(10)

,cust_id,date_id,age,gender,emp_index,channel_joined,cust_primary_type,cust_rel_type_at_start_month,activity_index,gross_income,...,pensions_plan,loans,taxes,credit_card,securities,home_acc,payroll,pensions,direct_debit,owned_products
6067945,1044707,20150928,32.0,M,N,KHE,1.0,I,0.0,134254.318238,...,0,0,0,0,0,0,0.0,0.0,0,[2]
6259779,1373938,20150928,35.0,F,N,KHK,1.0,A,1.0,33677.520000,...,0,0,0,0,0,0,0.0,0.0,1,"[2, 23]"
4738426,937462,20150828,24.0,M,N,KHD,1.0,A,1.0,130693.920000,...,0,0,0,0,0,0,0.0,0.0,0,[2]
9580234,849411,20160128,32.0,M,N,KAT,1.0,A,0.0,97703.400000,...,0,0,0,0,0,0,0.0,0.0,0,[-1]
11475706,487039,20160328,47.0,F,N,KAT,1.0,A,1.0,177538.590000,...,0,0,0,0,0,0,1.0,1.0,1,"[4, 7, 8, 12, 14, 21, 22, 23]"
11693908,1012458,20160328,40.0,F,N,KFC,1.0,I,0.0,325178.220000,...,0,0,0,0,0,0,0.0,0.0,0,[-1]
5133549,1051262,20150828,43.0,M,N,KFC,1.0,I,0.0,192691.740000,...,0,0,0,0,0,0,0.0,0.0,0,[-1]
3718025,437633,20150628,43.0,F,N,KFC,1.0,A,0.0,123959.010000,...,0,0,0,0,0,0,0.0,0.0,0,[13]
33732,1040285,20150128,23.0,F,N,KHE,1.0,A,1.0,60111.150000,...,0,0,0,0,0,0,0.0,0.0,0,[4]
8950697,1367342,20151228,42.0,M,N,KFC,1.0,A,1.0,41000.220000,...,0,0,0,0,0,0,0.0,0.0,1,"[2, 23]"


In [226]:
# Step 5: Get the latest record for each customer

df_train_result = df_train_subset_cols_cleaned.loc[df_train_subset_cols_cleaned.groupby('cust_id')['date_id'].idxmax()]
df_test_result = df_test_subset_cols_cleaned.loc[df_test_subset_cols_cleaned.groupby('cust_id')['date_id'].idxmax()]

In [227]:
df_test_result.sample(5)

,cust_id,date_id,age,gender,emp_index,channel_joined,cust_primary_type,cust_rel_type_at_start_month,activity_index,gross_income,...,pensions_plan,loans,taxes,credit_card,securities,home_acc,payroll,pensions,direct_debit,owned_products
249876,1479820,20160628,24,F,N,KHQ,1,I,0,134087.870595,...,0,0,0,0,0,0,0,0,0,"[7, 11]"
74298,1263517,20160628,37,F,N,KFC,1,I,0,207638.730000,...,0,0,0,0,0,0,0,0,0,[2]
70636,1235209,20160628,38,M,N,KFC,1,A,1,134087.870595,...,0,0,0,0,1,0,0,0,0,[19]
736743,764144,20160628,41,F,N,KCC,1,A,1,356681.850000,...,0,0,0,0,0,0,0,0,0,[-1]
523375,445683,20160628,48,M,N,KFC,1,A,1,122188.080000,...,0,0,0,0,0,0,0,0,0,[-1]


In [228]:
# Retrieve details for customers (latest record - not for all dates)
cust_num = 1339667
customer_info = get_customer_details(df_test_result, cust_num)
customer_info

,cust_id,date_id,owned_products
365363,1339667,20160628,[22]


## Implement KNN for Product Recommendation

Overview of steps:
- Transform the dataset into a user-item matrix where rows represent customers and columns represent products. 
- Fit the KNN model (Use the KNN algorithm to find similar customers)
- Make recommendations for a given customer, find the K nearest neighbors (similar customers).

### Helper functions

In [249]:
def pre_process_categorical_data(data: pd.DataFrame, columns_to_encode: List[str]) -> pd.DataFrame:
    """
    Convert categorical data to numerical data using one-hot encoding.

    Parameters:
    - data: DataFrame
    - columns_to_encode: List of columns to one-hot encode

    Returns:
    - DataFrame with one-hot encoded columns
    """
    return pd.get_dummies(data, columns=columns_to_encode)


import pandas as pd

def get_binary_item_ownership_columns(df_: pd.DataFrame, owned_items_col: str = "owned_products", nb_items: int = 25) -> pd.DataFrame:
    """
    Takes a dataframe df_, looks at the owned_product col which contains a list of item IDs,
    and converts that into a set of nb_items binary ownership columns called `owns_product_<ID>`
    which. These ownership indicator columns are returned.
    """

    ## Insert dummy user that owns everything
    dummy_user_id = -999
    dummy_user = pd.DataFrame(data={owned_items_col: {dummy_user_id: list(range(nb_items))}})
    df_ = pd.concat([df_, dummy_user], ignore_index=False)

    # Explode the owned items column
    df_exploded = df_.explode(owned_items_col)

    # Get dummy columns for ownership
    ownership_cols = pd.get_dummies(df_exploded[owned_items_col].astype(int), prefix='owns_product')

    # Group by the original index and sum the dummy columns
    ownership_cols = ownership_cols.groupby(level=0).sum()

    # Drop -1 (none-item) if exists
    if 'owns_product_-1' in ownership_cols.columns:
        ownership_cols = ownership_cols.drop(columns=['owns_product_-1'])

    # Remove dummy user
    ownership_cols = ownership_cols.drop(index=dummy_user_id)

    return ownership_cols



### Pre-process Data for KNN
 - Handle categorical data
 - Scale features (scikit-learn standard scaler)

In [230]:
# Check column types for encoding purposes
# Set the train and test dtypes to be the same

df_train_dtypes = df_train_result.dtypes
df_test_dtypes = df_test_result.dtypes


# Find the common columns between train and test sets
common_columns = set(df_train_dtypes.index) & set(df_test_dtypes.index)

# Check for data type mismatches and correct them
for column in common_columns:
    if df_train_dtypes[column] != df_test_dtypes[column]:
        print(f"Data type mismatch in column '{column}':")
        print(f"  Train data type: {df_train_dtypes[column]}")
        print(f"  Test data type: {df_test_dtypes[column]}")
        # Convert the test column to the train column's data type
        df_test_result[column] = df_test_result[column].astype(df_train_dtypes[column])
        print(f"  Column '{column}' in test set converted to {df_train_dtypes[column]}")

# Optionally, you can check columns only present in train or test
train_only_columns = set(df_train_dtypes.index) - set(df_test_dtypes.index)
test_only_columns = set(df_test_dtypes.index) - set(df_train_dtypes.index)

if train_only_columns:
    print("Columns only in train set:")
    for column in train_only_columns:
        print(f"  {column} (data type: {df_train_dtypes[column]})")

if test_only_columns:
    print("Columns only in test set:")
    for column in test_only_columns:
        print(f"  {column} (data type: {df_test_dtypes[column]})")


Data type mismatch in column 'cust_primary_type':
  Train data type: float64
  Test data type: int64
  Column 'cust_primary_type' in test set converted to float64
Data type mismatch in column 'activity_index':
  Train data type: float64
  Test data type: int64
  Column 'activity_index' in test set converted to float64
Data type mismatch in column 'payroll':
  Train data type: float64
  Test data type: int64
  Column 'payroll' in test set converted to float64
Data type mismatch in column 'age':
  Train data type: float64
  Test data type: int64
  Column 'age' in test set converted to float64
Data type mismatch in column 'pensions':
  Train data type: float64
  Test data type: int64
  Column 'pensions' in test set converted to float64


### Create a new row for each element in the list for 'owned_items'

In [247]:
# Product codes start from 0
# There are 24 products and -1 which represents no products owned ie: 25 total product codes
expected_num_items = 25

In [250]:
# Get ownership cols
## train
ownership_cols_train = get_binary_item_ownership_columns(
    df_=df_train_result,
    owned_items_col="owned_products",
    nb_items=expected_num_items
)

# ownership_cols_train.sample(10)

In [253]:
## test
ownership_cols_test = get_binary_item_ownership_columns(
    df_=df_test_result,
    owned_items_col="owned_products",
    nb_items=expected_num_items
)

# ownership_cols_test.sample(5)


,owns_product_0,owns_product_1,owns_product_2,owns_product_3,owns_product_4,owns_product_5,owns_product_6,owns_product_7,owns_product_8,owns_product_9,...,owns_product_15,owns_product_16,owns_product_17,owns_product_18,owns_product_19,owns_product_20,owns_product_21,owns_product_22,owns_product_23,owns_product_24
102559,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
169856,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
314119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
156131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72149,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [261]:
# Replace old ownership col
## train
df_train_processed = df_train_result.drop(columns="owned_products")
df_train_processed = df_train_result.join(ownership_cols_train)

## test
df_test_processed = df_test_result.drop(columns="owned_products")
df_test_processed = df_test_result.join(ownership_cols_test)


# Fill nan ownership with all 0s
df_train_processed = df_train_processed.fillna({col: 0 for col in ownership_cols_train.columns})
df_test_processed = df_test_processed.fillna({col: 0 for col in ownership_cols_train.columns})

# df_train_processed.head(3)
# df_test_processed.sample(2)


,cust_id,date_id,age,gender,emp_index,channel_joined,cust_primary_type,cust_rel_type_at_start_month,activity_index,gross_income,...,owns_product_15,owns_product_16,owns_product_17,owns_product_18,owns_product_19,owns_product_20,owns_product_21,owns_product_22,owns_product_23,owns_product_24
462913,332449,20160628,68.0,F,N,KAG,1.0,I,0.0,134087.870595,...,0,0,0,0,1,0,0,0,0,0
730073,786013,20160628,39.0,F,N,KAA,1.0,I,0.0,134087.870595,...,0,0,0,0,0,0,0,0,0,0


### Handle categorical columns

In [268]:
unique_values = df_train_cleaned['cust_rel_type_at_start_month'].unique()
print(unique_values)

# gender, emp_index, channel_joined, cust_rel_type_at_start_month

['A' 'I' 'P' 'R' 'N']


In [ ]:
# Specify columns to one-hot encode
columns_to_encode = [
    'Overall_customer_status',
    'Credit_customer_status',
    'NPV_bucket',
    #'Gender',
    'MyWorld_customer_status',
    'Insurance_customer_status',
    'Investment_customer_status',
    'Industry',
]

In [231]:
# Create a new row for each element in the list for 'owned_products'
# This is done to avoid the TypeError: unhashable type: 'list'

#df_train_exploded = df_train_result.explode('owned_products')
#df_test_exploded = df_test_result.explode('owned_products')

In [232]:
#df_train_exploded.sample(3)

In [233]:
# Retrieve details for customers (latest record - not for all dates)
#cust_num = 1110686
#customer_info = get_customer_details(df_train_exploded, cust_num)
#customer_info

In [234]:
# Convert 'object' datatypes that can numeric to numeric datatypes before doing encoding

# df_train_exploded['cust_primary_type'] = pd.to_numeric(df_train_exploded['cust_primary_type'])
# df_test_exploded['cust_primary_type'] = pd.to_numeric(df_test_exploded['cust_primary_type'])

# # Verify the conversion
# print ("Train")
# print(df_train_exploded['cust_primary_type'].dtype)
# print(df_train_exploded['cust_primary_type'].unique())

# print ("\nTest")
# print(df_test_exploded['cust_primary_type'].dtype)
# print(df_test_exploded['cust_primary_type'].unique())

In [235]:
# df_train_exploded.info()

In [236]:
#df_train_exploded.info()

# unique_values = df_train_exploded['cust_rel_type_at_start_month'].unique()
# print(unique_values)

In [237]:
# # Specify columns to one-hot encode
# columns_to_encode = [
#     'gender',
#     'emp_index',
#     'channel_joined',
#     'cust_rel_type_at_start_month',
# ]

# # Convert categorical data to numerical data
# df_encoded_train = pre_process_categorical_data(df_train_exploded, columns_to_encode)
# df_encoded_test = pre_process_categorical_data(df_test_exploded, columns_to_encode)

# df_encoded_train.sample(5)
# #df_encoded_test.sample(5)
